In [1]:
%cd ..

c:\Users\ernes\Documents\ML Projects\ds-project-test\ds-project-test


In [2]:
%pwd

'c:\\Users\\ernes\\Documents\\ML Projects\\ds-project-test\\ds-project-test'

In [ ]:

from src.ds_project.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.ds_project.utils.utils import read_yaml, create_directories
from src.ds_project import logger
from urllib.request import urlretrieve
import shutil

2025-04-28 15:50:57 - INFO: __init__ - Logging setup complete.


In [7]:
# download a file from a url using curl
!curl -O http://www3.dsi.uminho.pt/pcortez/wine/winequality.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0curl: (6) Could not resolve host: www3.dsi.uminho.pt

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    directory_path: Path
    source_url: str
    local_data_file: Path
    raw_data_path: Path
    train_data_path: Path
    test_data_path: Path

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_path=CONFIG_FILE_PATH, 
                 params_path = PARAMS_FILE_PATH, 
                 schema_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        print(self.config)
        print(self.config['artifacts_dir'])
        
        # Create artifact directories if they don't exist
        if not Path(self.config['artifacts_dir']).exists():
            create_directories([self.config['artifacts_dir']])
            logger.info(f"Created artifacts directory at {self.config['artifacts_dir']}")
        else:
            logger.info(f"Artifacts directory already exists at {self.config['artifacts_dir']}")


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config['data_ingestion']
        create_directories([config['directory'], config['raw_data_path'], config['train_data_path'], config['test_data_path']], True)


        data_ingestion_config = DataIngestionConfig(
            directory_path= Path(config['directory']),
            source_url= config['source_url'],
            local_data_file= Path(config['local_data_file']),
            raw_data_path= Path(config['raw_data_path']),
            train_data_path= Path(config['train_data_path']),
            test_data_path= Path(config['test_data_path'])
        )
        return data_ingestion_config
    


In [28]:
import requests
import zipfile
import os
from src.ds_project import logger

In [66]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_url} to {self.config.local_data_file}")
            filename, headers = urlretrieve(url=self.config.source_url, 
                                            filename=self.config.local_data_file)
            logger.info(f"Data downloaded to {self.config.local_data_file}")
        else:
            logger.info(f"Data already exists at {self.config.local_data_file}. Skipping download.")
    

    def extract_data(self):
        with zipfile.ZipFile(self.config.raw_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.raw_data_path)
        print(f"Data extracted to {self.config.raw_data_path}")

    def setup_data(self):
        # move the data to the raw data path
        if not self.config.local_data_file in os.listdir(self.config.raw_data_path):
            shutil.move(self.config.local_data_file, self.config.raw_data_path)
            logger.info(f'Data moved to {self.config.raw_data_path}')

In [67]:
manager = ConfigurationManager()
data_ingestion_config = manager.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_data()
data_ingestion.setup_data()

2025-04-28 18:51:58 - INFO: utils - YAML file config\config.yaml loaded successfully.
2025-04-28 18:51:58 - INFO: utils - YAML file parameters.yaml loaded successfully.
2025-04-28 18:51:58 - INFO: utils - YAML file schema.yaml loaded successfully.
{'artifacts_dir': 'artifact/', 'data_ingestion': {'directory': 'artifact/data_ingestion', 'source_url': 'https://github.com/MainakRepositor/Datasets/blob/master/Titanic.csv', 'local_data_file': 'artifact/data_ingestion/Titanic.csv', 'unzip_dir': 'artifact/data_ingestion', 'raw_data_path': 'artifact/data_ingestion/raw_data', 'train_data_path': 'artifact/data_ingestion/train_data', 'test_data_path': 'artifact/data_ingestion/test_data'}}
artifact/
2025-04-28 18:51:58 - INFO: 1679058489 - Artifacts directory already exists at artifact/
2025-04-28 18:51:58 - INFO: utils - Directory artifact/data_ingestion created.
2025-04-28 18:51:58 - INFO: utils - Directory artifact/data_ingestion/raw_data created.
2025-04-28 18:51:58 - INFO: utils - Directory a